In [1]:
import pandas as pd
import numpy as np

np.random.seed(2025)

In [2]:
# 2. Categorias
dias_semana = ['seg', 'ter', 'qua', 'qui', 'sex']
segmentos = ['alto_valor', 'medio_valor', 'baixo_valor']
periodos_dia = ['manha', 'tarde', 'noite']

In [3]:
def create_dataset_contexto(date_start, date_end):
    # 1. Datas realistas ao longo de 6 meses
    date_range = pd.date_range(start=date_start, end=date_end, freq='D')

    list_dfs = []
    for date in date_range:
        size_sample = np.random.randint(low=500, high=10001)
        # print(size_sample)
        
        df_tmp = pd.DataFrame({
            'date': [date]*size_sample
        })

        raw_probs = np.random.rand(len(periodos_dia))
        probs = raw_probs / raw_probs.sum()
        
        df_tmp['periodo_dia'] = np.random.choice(periodos_dia, size=size_sample, p=probs)
        df_tmp['dia_semana'] = df_tmp['date'].dt.dayofweek.map({0: 'seg', 1: 'ter', 2: 'qua', 3: 'qui', 4: 'sex', 5: 'sab', 6: 'dom'})
        df_tmp['segmento'] = np.random.choice(segmentos, size=size_sample, p=[0.2, 0.5, 0.3])
        df_tmp['tempo_cliente'] = np.random.exponential(scale=365, size=size_sample).astype(int)
        df_tmp['qtd_cliques_30d'] = np.random.poisson(lam=2, size=size_sample)
        df_tmp['usou_app_hoje'] = np.random.choice([0, 1], size=size_sample, p=[0.6, 0.4])
        df_tmp['dias_desde_ultimo_clique'] = np.random.randint(0, 91, size=size_sample)

        list_dfs.append(df_tmp)

    return pd.concat(list_dfs,ignore_index=True)

def simular_recompensa(row):
    p = 0.02  # baseline reduzido

    # Impacto do segmento
    segmento_bonus = {
        'alto_valor': 0.20,
        'medio_valor': 0.10,
        'baixo_valor': 0.00
    }
    p += segmento_bonus.get(row['segmento'], 0)

    # Interação entre segmento e período do dia
    if row['segmento'] == 'alto_valor':
        if row['periodo_dia'] == 'manha':
            p += 0.10
        elif row['periodo_dia'] == 'tarde':
            p -= 0.03
    elif row['segmento'] == 'medio_valor':
        if row['periodo_dia'] == 'tarde':
            p += 0.07
    elif row['segmento'] == 'baixo_valor':
        if row['periodo_dia'] == 'noite':
            p += 0.08
        elif row['periodo_dia'] == 'manha':
            p -= 0.02

    # Engajamento anterior
    p += min(row['qtd_cliques_30d'] * 0.008, 0.1)

    # Uso do app no dia
    if row['usou_app_hoje']:
        p += 0.04

    # Dia da semana
    if row['dia_semana'] == 'sex':
        p -= 0.04
    elif row['dia_semana'] == 'seg':
        p += 0.02  # segundas são melhores

    # Tempo de cliente
    if row['tempo_cliente'] < 30:
        p -= 0.03
    elif row['tempo_cliente'] > 365:
        p += 0.02  # clientes antigos clicam mais

    # Último clique
    if row['dias_desde_ultimo_clique'] < 3:
        p += 0.04
    elif row['dias_desde_ultimo_clique'] > 30:
        p -= 0.02

    # Clipping
    p = max(0.001, min(p, 0.95))

    return int(np.random.rand() < p)
    
df = create_dataset_contexto(date_start="2024-01-01", date_end="2024-12-01")
df['reward'] = df.apply(simular_recompensa, axis=1)
df

,date,periodo_dia,dia_semana,segmento,tempo_cliente,qtd_cliques_30d,usou_app_hoje,dias_desde_ultimo_clique,reward
0,2024-01-01,tarde,seg,medio_valor,834,1,0,37,0
1,2024-01-01,noite,seg,medio_valor,37,1,0,86,0
2,2024-01-01,manha,seg,baixo_valor,163,3,0,61,0
3,2024-01-01,tarde,seg,medio_valor,854,1,1,69,0
4,2024-01-01,manha,seg,alto_valor,19,2,1,75,0
...,...,...,...,...,...,...,...,...,...
1760127,2024-12-01,tarde,dom,alto_valor,401,2,0,19,0
1760128,2024-12-01,manha,dom,medio_valor,241,2,0,11,0
1760129,2024-12-01,tarde,dom,baixo_valor,4,3,0,1,0
1760130,2024-12-01,tarde,dom,medio_valor,737,5,0,42,0


In [4]:
df.to_csv('../dataset/raw/dataset.csv', index=False)